---
title: Predicting Diabetes in Pima Indian Women Using Logistic Regression
author: "Inder Khera, Jenny Zhang, Jessica Kuo, Javier Martinez (alphabetically ordered)"
date: "2023/12/08"
jupyter: python3
format: 
    html:
        toc: true
        toc-depth: 3
    pdf:
        toc: true
        toc-depth: 3
bibliography: references.bib
execute:
  echo: false
  warning: false
editor: source
---

In [ ]:
import pandas as pd
from IPython.display import Markdown, display
from tabulate import tabulate
import numpy as np
from scipy.stats import loguniform

import altair as alt
import altair_ally as aly

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import (
    RandomizedSearchCV,
    cross_validate,
    cross_val_score,
    train_test_split,
)
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression

import pandera as pa
from deepchecks.tabular import Dataset
from deepchecks.tabular.checks import (
    ClassImbalance, 
    PercentOfNulls,
    OutlierSampleDetection,
    DataDuplicates,
    MixedDataTypes,
    FeatureLabelCorrelation, 
    FeatureFeatureCorrelation
)
from deepchecks.tabular.checks.data_integrity import PercentOfNulls

import warnings
import json
import logging
import pickle

In [ ]:
import pandas as pd
import numpy as np

df_original = pd.read_csv("../data/raw/diabetes.csv")
df = pd.read_csv("../data/processed/df")
dropped_obs = df_original.shape[0] - df.shape[0]

pearson_corr = df.corr(method = "pearson")
spearman_corr = df.corr(method = "spearman")

max_pearson_corr = pearson_corr.where(~np.eye(pearson_corr.shape[0], dtype = bool)).stack().max()
max_spearman_corr = spearman_corr.where(~np.eye(spearman_corr.shape[0], dtype = bool)).stack().max()
corr_skin_bmi = spearman_corr("SkinThickness", "BMI")

mean_cv_score = pd.read_csv("../results/tables/mean_cv_score.csv")
mean_scores = pd.read_csv("../results/tables/mean_scores.csv")

value_counts_df = pd.read_csv("../results/tables/value_counts_df.csv")
fp_fn_df = pd.read_csv("../results/tables/fp_fn_df.csv")

accuracy_df = pd.read_csv("../results/tables/accuracy_df.csv")

coeff_table = pd.read_csv("../results/tables/coeff_table.csv")

pred_results_1_df =  pd.read_csv("../results/tables/pred_results_1_df.csv")

with open('../results/models/log_pipe.pkl', 'rb') as f:
    log_pipe = pickle.load(f)
with open('../results/models/random_fit.pkl', 'rb') as s:
    random_fit = pickle.load(s)

# Summary

This study evaluated logistic regression for predicting diabetes in Pima Indian women using features such as glucose levels, BMI, and pregnancies. 
The model achieved `{python} f"{accuracy_df['accuracy'].values[0] * 100:.0f}%"` accuracy on the test set, 
outperforming the baseline dummy classifier's `{python} f"{round(mean_cv_score['mean_cv_score'].values[0],2) * 100:.0f}%"`. 
Glucose was the most significant predictor, followed by BMI and pregnancies, while blood pressure and insulin had weaker impacts. 
The model misclassified `{python} f"{value_counts_df['misclassifications'].values[0]:,}"` cases, 
with `{python} f"{fp_fn_df['false negatives'].values[0]:,}"` false negatives and `{python} f"{fp_fn_df['false positives'].values[0]:,}"` false positives, 
highlighting areas for improvement.

The results indicate that logistic regression is a promising tool for diabetes screening, providing an efficient way to identify potential cases. 
However, the high number of false negatives is concerning, as they could lead to delayed diagnoses and treatments. 
Future improvements could include feature engineering to address misclassifications, testing alternative machine learning models, 
and incorporating additional data, such as lifestyle or genetic factors. 
Adding probability estimates for predictions could also enhance its clinical usability by helping prioritize further diagnostic tests. 
These steps could make the model more reliable and practical for real-world healthcare applications.

# Introduction

Diabetes is a serious chronic disease characterized by high levels of glucose in the blood, 
caused by either insufficient insulin production by the pancreas or the body’s inability to effectively use insulin. 
It has become a significant global health issue, with its prevalence nearly doubling since 1980, and in 2022, 
14% of adults aged 18 and older were diagnosed with diabetes, doubling from 7% in 1990 [@who_diabetes]. 
Diabetes can lead to severe complications, including blindness, kidney failure, heart attacks, strokes, and lower limb amputations. 
Early detection enables timely interventions, reduces complications, lowers healthcare costs, 
and improves quality of life and long-term outcomes [@marshall2006prevention].

Artificial intelligence (AI) leverages computer systems and big data to simulate intelligent behavior with minimal human intervention, 
and within it, machine learning (ML) is a subset of AI methodologies. Since the rise of AI, 
Machine learning has increasingly been applied in various areas of disease detection and prevention in the healthcare field [@bini2018artificial]. 
Numerous machine learning techniques have been deployed to develop more efficient 
and effective methods for diagnosing chronic diseases [@battineni2020applications]. 
Utilizing machine learning methods in diabetes research has been proven to be a critical strategy 
or harnessing large volumes of diabetes-related data to extract valuable insights [@agarwal2022machine]. 
Therefore, The goal of this report is to leverage a supervised machine learning model, logistic regression (LR), 
to evaluate its predictive performance in diagnosing diabetes using a real-world dataset focused specifically on Pima Indian women aged 21 and older.

# Methods and Results

### Data

The dataset that was used for the analysis of this project was created by Jack W Smith, JE Everhart, WC Dickson, WC Knowler, RS Johannes 
and sourced from the National Librabry of Medicine database from the National Institues of Health. 
Access to their respective analysis can be found [here](https://pmc.ncbi.nlm.nih.gov/articles/PMC2245318/) 
and access to the dataset can be found via [kaggle](https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database/data) [@dua2017pima]. 
The primary objective of the dataset is to enable diagnostic prediction of whether a patient has diabetes based on specific diagnostic measurements. 
To ensure consistency and relevance, several constraints were applied to the selection of data instances. 
Specifically, the dataset includes only female patients who are at least 21 years old and of Pima Indian heritage.

Each row/obersvation from the dataset is an individual that identifies to be a part of the Pima (also known as The Akimel O'odham) Indeginous group, 
located mainly in the Central and Southern regions of the United States. Each observation recorded has summary statistics regarding features 
that include the Age, BMI, Blood Pressure, Number of Pregnancies, as well as 
The Diabetes Pedigree Function (which is a score that gives an idea about how much correlation is between person with diabetes and their family history). 
The dataset offers comprehensive features for machine learning analysis.

### Analysis

Logistic Regression was employed to develop a classification model for predicting whether the patient is diabetic or not 
(as indicated in the `outcome` column of the dataset). All variables from the original dataset were used to train the model. 
The data was split into 70% for the training set and 30% for the testing set. Hyperparameter tuning was performed using `RandomizedSearchCV`, 
with the accuracy score serving as the classification metric. All variables were standardized just before model fitting. 
The analysis was conducted using the Python programming language [@Python] and several Python packages: numpy [@harris2020array], 
Pandas [@pandas], altair [@altair], altair_ally [@ostblom2021altair] and scikit-learn [@pedregosa2011scikit]. 
The code used for this analysis and report is available at: https://github.com/UBC-MDS/diabetes_predictor_py


In [ ]:
# load data
df_original = pd.read_csv('../data/raw/diabetes.csv')
df_original

In [ ]:
# Configure logging
logging.basicConfig(
    filename="validation_errors.log",
    filemode="w",
    format="%(asctime)s - %(message)s",
    level=logging.INFO,
)

# Define schema
schema = pa.DataFrameSchema(
    { 
        "Outcome": pa.Column(int, pa.Check.isin([0, 1])),
        "Pregnancies": pa.Column(int, pa.Check.between(0, 15), nullable=True),
        "Glucose": pa.Column(int, pa.Check.between(50, 240), nullable=True),
        "BloodPressure": pa.Column(int, pa.Check.between(40, 180), nullable=True),
        "SkinThickness": pa.Column(int, pa.Check.between(0, 80), nullable=True),
        "Insulin": pa.Column(int, pa.Check.between(0, 800), nullable=True),
        "BMI": pa.Column(float, pa.Check.between(0, 65), nullable=True),
        "DiabetesPedigreeFunction": pa.Column(float, pa.Check.between(0, 2.5), nullable=True),
        "Age": pa.Column(int, pa.Check.between(18, 90), nullable=True),
    },
    checks=[
        pa.Check(lambda df: ~df.duplicated().any(), error="Duplicate rows found."),
        pa.Check(lambda df: ~(df.isna().all(axis=1)).any(), error="Empty rows found.")
    ],
    drop_invalid_rows=False,  # Ensure this is properly closed
)

# Initialize error cases DataFrame
error_cases = pd.DataFrame()
data = df_original.copy()

# Validate data and handle errors
try:
    validated_data = schema.validate(data, lazy=True)
except pa.errors.SchemaErrors as e:
    error_cases = e.failure_cases

    # Convert the error message to a JSON string
    error_message = json.dumps(e.message, indent=2)
    logging.error("\n" + error_message)

# Filter out invalid rows based on the error cases
if not error_cases.empty:
    invalid_indices = error_cases["index"].dropna().unique()
    df = (
        data.drop(index=invalid_indices)
        .reset_index(drop=True)
        .drop_duplicates()
        .dropna(how="all")
    )
else:
    df = data

In [ ]:
df

df.to_csv('../data/processed/df.csv')

In [ ]:
# EDA
print(df_original.shape)
print(df.shape)

We do see `{python} f"{dropped_obs:,}"` observations being dropped post our preliminary data validation as the rows dropped contain meaningless 
and / or invalid data entries that would unlikely provide useful information but instead introduce noise or spurious relationships into our model. 
Details of the observations dropped can be found at https://github.com/UBC-MDS/diabetes_predictor_py under `validation_errors.log`. 
It is clearly shown from the log that majority of data points that are dropped contained values of 0 where levels cannot possibly be, 
hence, we suspect that the values recorded at 0 are likely missing values recorded in such way, 
a critical point we will confirm with data collectors if we have access. 


In [ ]:
# EDA
df.info()

In [ ]:
# Create the split
train_df, test_df = train_test_split(df,
                                     train_size = 0.7, 
                                     random_state=123)

df.to_csv('../data/processed/train_df.csv')
df.to_csv('../data/processed/test_df.csv')

In [ ]:
# Explore training data
census_summary = train_df.describe()
census_summary

In [ ]:
# List features
features = census_summary.columns.tolist()
features

![Comparison of the empirical distributions of training data predictors between those non-diabetic and diabetic.](../results/figures/feature_histograms.png){#fig-feature_histograms width=80%}

@fig-feature_histograms above illustrates the distribution of each feature, categorized based on the Outcome variable: 0 (Non-Diabetic) and 1 (Diabetic). 
This visualization provides insights into the relationships between individual features and the target variable.

For the `Glucose` levels, Non-Diabetic class exhibits a roughly normal distribution, 
whereas the Diabetic class shows a pronounced shift toward the middle-to-higher range of glucose levels. 

The `BMI` distribution resembles a normal distribution but skews slightly toward higher values. 
Similar to Glucose levels, the Diabetic class displays a decent shit towards the middle-to-higher ranges when compared to Non-Diabetic class, 
suggesting the potential for distinct differences between target groups within this category.

The `Age` distribution reveals that individuals aged 20 to 32 are predominantly Non-Diabetic. 
Beyond age 32, the counts of Diabetic and Non-Diabetic individuals become comparable, 
with some bins showing a higher count for the Diabetic class, despite fewer overall observations in this group. 
The Non-Diabetic class leans toward younger ages, while the Diabetic class has a more even distribution across its age range.

For `Pregnancies`, `Insulin`, and `DiabetesPedigreeFunction` -genetic risk of diabetes based on family history ranging from 0 to 2.5, 
the lower range of pregnancies is dominated by the Non-Diabetic class, whereas whereas higher numbers are more common in the Diabetic class.

For `Blood Pressure` and `Skin Thickness`, both the Diabetic and Non-Diabetic classes approximates a normal distribution; 
however, the Non-Diabetic distribution skews slightly towards lower values, while the Diabetic class skews more towards higher values.


In [ ]:
# validate training data for class imbalance for target variable 
# Do these on training data as part of EDA! 
train_df_ds = Dataset(train_df, label = 'Outcome', cat_features=[])

check_lab_cls_imb = ClassImbalance().add_condition_class_ratio_less_than(0.4)
check_lab_cls_imb_result = check_lab_cls_imb.run(dataset = train_df_ds)

if check_lab_cls_imb_result.passed_conditions():
    raise ValueError("Class imbalance exceeds the maximum acceptable threshold.")

In [ ]:
# validate training data for percent of nulls
check_pct_nulls = PercentOfNulls().add_condition_percent_of_nulls_not_greater_than(0.05)
check_pct_nulls_result = check_pct_nulls.run(dataset = train_df_ds)

if not check_pct_nulls_result.passed_conditions():
    raise ValueError("Percent of nulls exceeds the maximum acceptable threshold for at least one column.")

In [ ]:
# validate training data for percent of outlier samples using loOP algo
check_out_sample = (
    OutlierSampleDetection(nearest_neighbors_percent = 0.01, extent_parameter = 3)
    .add_condition_outlier_ratio_less_or_equal(max_outliers_ratio = 0.001, outlier_score_threshold = 0.9)
)
check_out_sample_result = check_out_sample.run(dataset = train_df_ds)

if not check_out_sample_result.passed_conditions():
    raise ValueError("Number of outlier samples exceeds the maximum acceptable threshold.")

In [ ]:
# validate training data for data duplicates
# set duplicate condition to 0 as would not expect any two patient with the exact same situation
check_data_dup = DataDuplicates().add_condition_ratio_less_or_equal(0)
check_data_dup_result = check_data_dup.run(dataset = train_df_ds)

if not check_data_dup_result.passed_conditions():
    raise ValueError("Data duplicates exceed the maximum acceptable threshold.")

In [ ]:
# validate training data for mixed data types across all columns
check_mix_dtype = MixedDataTypes().add_condition_rare_type_ratio_not_in_range((0.01, 0.2))
check_mix_dtype_result = check_mix_dtype.run(dataset = train_df_ds)

if not check_mix_dtype_result.passed_conditions():
    # raise a warning instead of an error in this case
    warnings.warn("Percentage of rare data type in dangerous zone for at least one column")

![Pearson and Spearman correlations across all features.](../results/figures/correlation_heat_map.png){#fig-correlation_heat_map width=80%}

@fig-correlation_heat_map shows the correlation between all of the respective features. 
The main reasoning to analyze this is to see if there is any multicollinearity between any of the features 
which could be problamatic when conducting a Logistic Regression. We see that highest level of correlation 
is between Age and Pregnancies (`{python} f"{max_spearman_corr:.00f}"` via Spearman, and `{python} f"{max_pearson_corr:.00f}"` by Pearson). Since this is below the threshold of 0.7, 
we can conclude that all features' coefficients are suitable and will not cause any multicollinearity in our model. 

![Pairwise scatterplots between each of features in dataset to visualize relationship.](../results/figures/pairwise_scatterplot.png){#fig-pairwise_scatterplot width=120%}

@fig-pairwise_scatterplot illustrates the relationships between the features. For the most part, the features do not display noticeable trends. 
However, Skin Thickness and BMI show a moderate visual relationship, 
which is intuitive since higher body mass is generally associated with increased skin thickness. 

Referring back to the correlation graph, Skin Thickness and BMI have a Spearman correlation of `{python} f"{corr_skin_bmi:.00f}"`. 
This value is below the multicollinearity threshold of 0.7, indicating that these features do not pose a risk of multicollinearity in our model.


In [ ]:
# validate training data for anomalous correlations between target/response variable 
# and features/explanatory variables, 
# as well as anomalous correlations between features/explanatory variables

check_feat_lab_corr = FeatureLabelCorrelation().add_condition_feature_pps_less_than(0.7)
check_feat_lab_corr_result = check_feat_lab_corr.run(dataset = train_df_ds)

check_feat_feat_corr = FeatureFeatureCorrelation().add_condition_max_number_of_pairs_above_threshold(threshold = 0.7, n_pairs = 0)
check_feat_feat_corr_result = check_feat_feat_corr.run(dataset = train_df_ds)

if not check_feat_lab_corr_result.passed_conditions():
    raise ValueError("Feature-Label correlation exceeds the maximum acceptable threshold.")

if not check_feat_feat_corr_result.passed_conditions():
    raise ValueError("Feature-feature correlation exceeds the maximum acceptable threshold.")

We then further split our dataset into X and y for both the training and test. 


In [ ]:
X_train = train_df.drop(columns = ['Outcome'])
y_train = train_df['Outcome']
X_test = test_df.drop(columns = ['Outcome'])
y_test = test_df['Outcome']

As a result, the Dummy Classifier acts as our baseline for conductin our initial analysis. The Dummy Baseline gives us a score of around `{python} round(mean_cv_score['mean_cv_score'].values[0], 2)`.

We will use a Logistic Regression model for classification. Given the presence of outliers in our features, it is advisable to apply StandardScaler() to normalize the feature values before fitting the model. This ensures that all features are on a similar scale, improving the model's performance and stability.


In [ ]:
# Create Logistic Regression pipeline
log_pipe=make_pipeline(
    StandardScaler(), 
    LogisticRegression(max_iter=2000,random_state=123)
)

We optimize the hyperparameter `C` for our Logistic Regression model using a random search approach.


In [ ]:
# Hyperparameter optimization
np.random.seed(123)
param_dist = {
    "logisticregression__C": loguniform(1e-5, 1e+5) 
}

In [ ]:
# Create Random Search
random_search = RandomizedSearchCV(log_pipe,param_dist,
                                   n_iter=20,
                                   n_jobs=-1,
                                   return_train_score=True,
                                   random_state=123)

random_search.fit(X_train,y_train)

Identify the optimal value for the hyperparameter C to be used in our Logistic Regression model.


In [ ]:
# Identify optiomized parameter C
best_params = random_search.best_params_ 
best_params

In [ ]:
# Display scores
pd.DataFrame(random_search.cv_results_).sort_values(
    "rank_test_score").head(3)[["mean_test_score",
                                "mean_train_score"]]

Having determined the best Logistic Regression model for our analysis, we further explore feature importance with coefficients.


In [ ]:
# Best model from the search
best_model = random_search.best_estimator_

# Retrieve the coefficients and feature names
coefficients = best_model.named_steps['logisticregression'].coef_.flatten()
features = X_train.columns  

# Create a DataFrame to display the feature names and corresponding coefficients
coeff_df = pd.DataFrame({
    'Features': features,
    'Coefficients': coefficients
})

# Sort by 'Coefficients' in descending order to see the most important features first
coeff_df_sorted = coeff_df.sort_values(by = 'Coefficients', ascending = False)

# Create a heatmap for the coefficients (we will visualize them as a single column)
coeff_df_sorted.style.format(
    precision = 3
).background_gradient(
    axis = None
)

In [ ]:
#| label: tbl-coeff_table
#| tbl-cap: Logistic regression feature importance measured by coefficients.
#| echo: false

Markdown(coeff_table.to_markdown(index = False))

Based on the heatmap and @tbl-coeff_table above, the feature importance coefficients for the logistic regression model predicting diabetes reveal that `Glucose` (`{python} round(coeff_table.iloc[coeff_table[coeff_table.iloc[:, 0] == 'Glucose'].index[0], 1], 2)`) is the strongest positive influence, followed by `BMI` (`{python} round(coeff_table.iloc[coeff_table[coeff_table.iloc[:, 0] == 'BMI'].index[0], 1], 2)`), `Pregnancies` (`{python} round(coeff_table.iloc[coeff_table[coeff_table.iloc[:, 0] == 'Pregnancies'].index[0], 1], 2)`), `Age` (`{python} round(coeff_table.iloc[coeff_table[coeff_table.iloc[:, 0] == 'Age'].index[0], 1], 2)`), and `DiabetesPedigreeFunction` (`{python} round(coeff_table.iloc[coeff_table[coeff_table.iloc[:, 0] == 'DiabetesPedigreeFunction'].index[0], 1], 2)`). The negative influence `SkinThickness` (`{python} round(coeff_table.iloc[coeff_table[coeff_table.iloc[:, 0] == 'SkinThickness'].index[0], 1], 2)`) along with the remaining positive features `BloodPressure` (`{python} round(coeff_table.iloc[coeff_table[coeff_table.iloc[:, 0] == 'BloodPressure'].index[0], 1], 2)`) and `Insulin` (`{python} round(coeff_table.iloc[coeff_table[coeff_table.iloc[:, 0] == 'Insulin'].index[0], 1], 2)`), have weak impacts on the prediction, with their effects being less pronounced. 

We then evaluate the best Logistic Regression model, obtained from the hyperparameter search, on the test set.


In [ ]:
# Make predictions using the best model== "Glucose"]["Coefficient"].values[0]
y_pred = best_model.predict(X_test)

In addition, to enhance the model's practical use in a clinical setting, we are providing and reporting probability estimates for the predictions of diabetes. Offering probability estimates would allow clinicians to gauge the model's confidence in its predictions. This would give clinicians the opportunity to conduct additional diagnostic tests if the predicted probability for the outcome (i.e. diagnosis of prediction) is not sufficiently high.


In [ ]:
#| label: pred_results_1_table
#| tbl-cap: Prediction Results with Probability Estimates for Diabetes Classification.
#| echo: false

Markdown(pred_results_1_df.head().to_markdown(index = False))

In [ ]:
y_pred_prob = best_model.predict_proba(X_test)
pred_bool = (y_test == y_pred)
pred_results_1 = np.vstack([y_test, y_pred, pred_bool, y_pred_prob[:, 1]])
pred_results_1_df = pd.DataFrame(pred_results_1.T, 
                                 columns = ['y_test', 'y_pred', 'pred_bool', 'y_pred_prob_1'])
pred_results_1_df['pred_bool'] = pred_results_1_df['pred_bool'] == 1
pred_results_1_df.head()

Our prediction model performed decent on test data, with a final overall accuracy of `{python} accuracy_df['accuracy'].values[0]`. In addition, looking through the prediction results dataframe, there are a total of `{python} value_counts_df['misclassifications'].values[0]` mistakes. Of which, `{python} fp_fn_df['false negatives'].values[0]` mistakes were predicting diabetic as non-diabetic (false negatives) and `{python} fp_fn_df['false positives'].values[0]` mistakes were made predicting diabetic as non-diabetic (false positives). Considering implementation in clinic, there is room for improvement in the algorithm as false negatives are more harmful than false positives, and we should aim to lower false positives even further.

Moreover, visualizing prediction probabilities alongside the prediction accuracy for each test sample provides a clearer understanding of the model's performance. This approach allows us to easily assess how well the model predicts, while also highlighting patients who were misdiagnosed. Particularly, it helps us focus on false negatives, as the consequences of these errors are more critical in a clinical context.

![Test Set Prediction Accuracy by Prediction Probability.](../results/figures/predic_chart.png){#fig-test_set_pred_accur width=80%}

# Discussion

While the performance of this model may be valuable as a screening tool in a clinical context, especially given its improvements over the baseline, there are several opportunities for further enhancement. One potential approach is to closely examine the `{python} value_counts_df['misclassifications'].values[0]` misclassified observations, comparing them with correctly classified examples from both classes. The objective would be to identify which features may be contributing to the misclassifications and investigate whether feature engineering could help the model improve its predictions on the observations it is currently struggling with. Additionally, we would try seeing whether we can get improved predictions using other classifiers. Other classifiers we might try are 1) random forest because it automatically allows for feature interaction, 2) k-nearest neighbours (k-NN) which usually provides easily interpretable and decent predictions, and 3) support vector classifier (SVC) as it allows for non-linear prediction using the rbf kernel. Finally, there runs the possibility that the features offered from this dataset alone are not sufficient to predict with high accuracy. In this case, conducting additional conversations with data collectors for additional useable information or explore additional datasets that can be joined so our set of features can be expanded for more complicated analysis might be beneficial. 

At last, we recognize the limitation with this dataset, as it focuses solely on Pima Indian women aged 21 and older, which limits its generalizability to other populations. To improve the analysis, it would be valuable to combine this data with other datasets representing different age groups, genders, and ethnicities, enabling more comprehensive insights and broader applicability of the findings.

# Conclusion

In conclusion, this study demonstrated the effectiveness of logistic regression in predicting diabetes among Pima Indian women using diagnostic features such as glucose, BMI, and pregnancies. With an accuracy of `{python} f"{accuracy_df['accuracy'].values[0] * 100:.0f}%"` on the test set, the model outperformed the baseline Dummy Classifier's `{python} f"{round(mean_cv_score['mean_cv_score'].values[0],2) * 100:.0f}%"`. Glucose was identified as the most influential predictor, followed by BMI and pregnancies, while features like blood pressure, insulin, and skin thickness had weaker impacts. However, the model's `{python} value_counts_df['misclassifications'].values[0]` misclassifications, particularly the `{python} fp_fn_df['false negatives'].values[0]` false negatives, underscore the need for further refinement to minimize the risk of undiagnosed cases.

These findings highlight logistic regression's potential as an initial screening tool in clinical settings, offering a data-driven approach to early diabetes detection. Nevertheless, improvements are essential to enhance its accuracy and practical utility. Strategies such as feature engineering, alternative machine learning models, and the incorporation of additional data, such as lifestyle or genetic factors, could further optimize performance. Additionally, providing probability estimates for predictions could enhance clinical decision-making by identifying cases requiring further diagnostics. With these refinements, the model could become a valuable tool for reducing complications and improving outcomes in diabetes care.

# References
